In [1]:
import numpy as np

from Resources.FDS.PeriodicTableFDS653 import PeriodicTable as PT
from Resources.ChemicalSpecies import ChemicalSpecies as CS

In [2]:
# Define hydrogen atomic fraction in soot.
X_H = 0.1


# Define CO and soot yields.
y_CO = 0
y_Soot = 0.01

In [3]:
print(PT['C'][1], 
      PT['H'], 
      PT['O'], 
      PT['N'], 
      PT['Cl'])

print(CS['C6H5CH3'])

12.0107 ['HYDROGEN', 1.00794] ['OXYGEN', 15.9994] ['NITROGEN', 14.0067] ['CHLORINE', 35.453]
['TOLUENE', [['C', 7], ['H', 8]]]


In [4]:
# define atomic weights
W_C = PT['C'][1]
W_H = PT['H'][1]
W_O = PT['O'][1]
W_N = PT['N'][1]
W_Cl = PT['Cl'][1]

In [5]:
# define primitive species
# 'methane','ethylene','chlorine','oxygen','nitrogen','water vapor',
# 'carbon dioxide','carbon monoxide','hydrochloric acid','soot'
i_toluene         = 0
i_oxygen          = 1
i_nitrogen        = 2
i_water_vapor     = 3
i_carbon_dioxide  = 4
i_carbon_monoxide = 5
i_soot            = 6

n_species = 7


# define the element matrix (number of atoms [rows] for each primitive species
# [columns])
#              0  1  2  3  4  5  6
E = np.array([[7, 0, 0, 0, 1, 1, (1-X_H)],  # C
              [8, 0, 0, 2, 0, 0, X_H    ],  # H
              [0, 2, 0, 1, 2, 1, 0      ],  # O
              [0, 0, 2, 0, 0, 0, 0      ]])  # N

print('Element matrix')
print(E)

Element matrix
[[7.  0.  0.  0.  1.  1.  0.9]
 [8.  0.  0.  2.  0.  0.  0.1]
 [0.  2.  0.  1.  2.  1.  0. ]
 [0.  0.  2.  0.  0.  0.  0. ]]


In [6]:
# primitive species molecular weights
A = np.array([W_C, W_H, W_O, W_N])
W = np.dot(E.T, A)

print('Primitive species molecular weights')
print(W)

Primitive species molecular weights
[92.13842  31.9988   28.0134   18.01528  44.0095   28.0101   10.910424]


In [7]:
# define the mass fractions of the background air (grab from FDS .out file)
v_1 = np.zeros((n_species))
v_1[i_oxygen]         = 0.231181
v_1[i_nitrogen]       = 0.763077
v_1[i_water_vapor]    = 0.005149
v_1[i_carbon_dioxide] = 0.000592
print('Mass fractions of AIR')
print(v_1, '\n')

Mass fractions of AIR
[0.00000e+00 2.31181e-01 7.63077e-01 5.14900e-03 5.92000e-04 0.00000e+00
 0.00000e+00] 



In [8]:
# convert to volume fractions
#
# use these in new FDS input file, else atom balance error may
# occur!
W_1 = 1./np.sum(v_1/W)
print('Molecular weight of AIR')
print(W_1, '\n')

v_1 = v_1*W_1/W
v_1 = v_1/np.sum(v_1)
print('Volume fractions of AIR')
print(v_1, '\n')  

Molecular weight of AIR
28.76567445926183 

Volume fractions of AIR
[0.00000000e+00 2.07822712e-01 7.83568741e-01 8.22160176e-03
 3.86945529e-04 0.00000000e+00 0.00000000e+00] 



In [9]:
# define volume fractions of fuel mixture (assumed known)
v_2 = np.zeros((n_species))
v_2[i_toluene]        = 100.0
v_2 = v_2/np.sum(v_2)
print('Volume fractions of FUEL')
print(v_2, '\n')

W_2 = np.sum(v_2*W)
print('Molecular weight of FUEL')
print(W_2, '\n')

Volume fractions of FUEL
[1. 0. 0. 0. 0. 0. 0.] 

Molecular weight of FUEL
92.13842 



In [22]:
# the reaction coefficients for the product primitive species temporarily
# stored in v_3
v_3 = np.zeros((n_species))


# compute what we know so far 
# (meaning the amount of elements known 
# to be consumed by the yields)
v_3[i_carbon_monoxide] = W_2/W[i_carbon_monoxide]*y_CO
v_3[i_soot]            = W_2/W[i_soot]*y_Soot

print('v_2', 'v_3')
print(v_2)
print(v_3)
print()

# linear system right hand side
b = np.dot(E, v_2-v_3)
print('b')
print(b, '\n')

v_2 v_3
[1. 0. 0. 0. 0. 0. 0.]
[0.        0.        0.        0.        0.        0.        0.0844499]

b
[6.92399509 7.99155501 0.         0.        ] 



In [11]:
# matrix
L = np.array([np.dot(E, v_1),
              E[:, i_carbon_dioxide],
              E[:, i_water_vapor],
              E[:, i_nitrogen]]).T
print('L')
print(L, '\n')

L
[[3.86945529e-04 1.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.64432035e-02 0.00000000e+00 2.00000000e+00 0.00000000e+00]
 [4.24640916e-01 2.00000000e+00 1.00000000e+00 0.00000000e+00]
 [1.56713748e+00 0.00000000e+00 0.00000000e+00 2.00000000e+00]] 



In [12]:
# % solve the system
x = np.linalg.solve(L, b)
print('x')
print(x, '\n')

x
[-42.93026383   6.94060676   4.34873304  33.63881278] 



In [13]:
nu_1                  = x[0]  # background stoichiometric coefficient
v_3[i_carbon_dioxide] = x[1]
v_3[i_water_vapor]    = x[2]
v_3[i_nitrogen]       = x[3]

# print('Products')
# print(v_3, '\n')

nu_2 = -1        # fuel stoichiometric coefficient
nu_3 = sum(v_3)  # prod stoichiometric coefficient

v_3 = v_3/nu_3  # normalized product volume fractions

print('Products')
print(v_3, '\n')

# check mass balance (should be 0)
print('Mass balance')
print(nu_1*np.sum(v_1*W) + nu_2*np.sum(v_2*W) + nu_3*np.sum(v_3*W), '\n')

Products
[0.         0.         0.74731988 0.09661146 0.15419252 0.
 0.00187614] 

Mass balance
2.2737367544323206e-13 



In [14]:
# print lumped reaction coefficients
print('Lumped species reaction coefficients')
print('nu_1 = ' + str(nu_1))
print('nu_2 = ' + str(nu_2))
print('nu_3 = ' + str(nu_3), '\n')

Lumped species reaction coefficients
nu_1 = -42.930263833181655
nu_2 = -1
nu_3 = 45.012602484273486 



In [15]:
# Calculation of stoichiometry data for FDS input files. 
# Fuel is expected to be a list of the following structure: [C,H,O,N,CL]. 
# Elements in this list are coefficients (float) of the chemical formula.
# Fuel label is expected to be a string, will be used for the REAC_ID.
# Various yields can be specified, as well as the percentage of hydrogen 
# in the soot.
def stoichiometry(fuel,
                  fuel_label,
                  oxydizer_label,
                  product_label,
                  co2_yield=0.0,
                  co_yield=0.0,
                  soot_yield=0.0,
                  soot_h_fraction=0.0,
                  simple_air=False):
    
    # Calculate the molecular weight of the fuel molecule, in g/mol.
    w_fuel = fuel[0] * cMolWeight + \
             fuel[1] * hMolWeight + \
             fuel[2] * oMolWeight + \
             fuel[3] * nMolWeight + \
             fuel[4] * clMolWeight
    
    # Calculate the molecular weight of the soot, in g/mol.
    w_s = soot_h_fraction * hMolWeight + (1 - soot_h_fraction) * cMolWeight
    print('Soot weight:')
    print(w_s)
    
    # Calculate the stoichiometric coefficient of nitrogen; gas (products).
    # Nu means the greek letter.
    nu_n2 = fuel[3]/2
    print('Nu N2:')
    print(nu_n2)
    
    # Calculate the stoichiometric coefficient of soot; solid (products).
    # nu_s = w_fuel / w_s * soot_yield
    nu_s = stoichiometric_coefficient(w_fuel, w_s, soot_yield)
    print('Nu soot:')
    print(nu_s)
    
    # Calculate the stoichiometric coefficient of 
    # carbon monoxide; gas (products).
    # nu_co = w_fuel / co_mol_weight * co_yield
    nu_co = stoichiometric_coefficient(w_fuel, co_mol_weight, co_yield)
    print('Nu CO:')
    print(nu_co)
    
    # Calculate the stoichiometric coefficient of 
    # hydrogen chloride; gas (products).
    nu_hcl = fuel[4]
    # nu_hcl = stoichiometric_coefficient(w_fuel, clMolWeight, hclYield)
    print('Nu HCl:')
    print(nu_hcl)
    
    # Calculate the stoichiometric coefficient of water; gas (products).
    nu_h2o = fuel[1]/2 - soot_h_fraction / 2 * nu_s - fuel[4]
    print('Nu H2O:')
    print(nu_h2o)
    
    # Calculate the stoichiometric coefficient of 
    # carbon dioxide; gas (products).
    nu_co2 = fuel[0] - nu_co - (1 - soot_h_fraction) * nu_s
    print('Nu CO2:')
    print(nu_co2)
    
    # Calculate the stoichiometric coefficient of 
    # oxygen; gas (reactants).
    nu_o2 = nu_co2 + nu_co/2 + nu_h2o/2 - fuel[2]/2
    print('Nu O2:')
    print(nu_o2)

    # Volume fraction
    # Simple air follows the simplification that air consists out of 21 % 
    # of oxygen and 79 % of nitrogen.
    if simple_air is False:
        
        # The air_demand is based on the amount of oxygen needed for the
        # combustion. All the other components are scaled according to this 
        # value.
        air_demand = nu_o2 / airVolFrac[1]
        print('Air demand:')
        print(air_demand)
        print('nu_o2:')
        print(nu_o2)
        print('airVolFrac[1]:')
        print(airVolFrac[1])

        prod_n2 = air_demand * airVolFrac[0] + nu_n2
        print('Product N2:')
        print(prod_n2)
        
        prod_h2o = air_demand * airVolFrac[2] + nu_h2o
        print('product H2O:')
        print(prod_h2o)
        
        prod_co2 = air_demand * airVolFrac[3] + nu_co2
        print('Product CO2:')
        print(prod_co2)

    elif simple_air is True:
        air_demand = nu_o2 / airVolFracSimple[1]
        print('Air demand:')
        print(air_demand)

        prod_n2 = air_demand * airVolFracSimple[0] + nu_n2
        print('Product N2:')
        print(prod_n2)
        
        prod_h2o = nu_h2o
        print('product H2O:')
        print(prod_h2o)
        
        prod_co2 = nu_co2
        print('Product CO2:')
        print(prod_co2)
        pass
    else:
        print('Parameter "simple_air" needs to be True or False (bool)!')
        
    prod_hcl = nu_hcl
    print('Product HCl:')
    print(prod_hcl)
    
    prod_soot = nu_s
    print('Product soot:')
    print(prod_soot)
    
    prod_co = nu_co
    print('Product CO:')
    print(prod_co)
    
    # Calculate the total molar amount of all the products to normalise them.
    total_product_mol = (prod_n2 + prod_co2 + prod_h2o +
                         prod_hcl + prod_co + prod_soot)
    
    products = ['N2: '+str(prod_n2),
                'CO2: '+str(prod_co2),
                'H2O: '+str(prod_h2o),
                'HCl: '+str(prod_hcl),
                'CO: '+str(prod_co),
                'Soot: '+str(prod_soot)]
    
    # Normalise products, used for lumped species.
    products_normalised = [prod_n2/total_product_mol,
                           prod_co2/total_product_mol,
                           prod_h2o/total_product_mol,
                           prod_hcl/total_product_mol,
                           prod_co/total_product_mol,
                           prod_soot/total_product_mol]
    print(products_normalised)

    # return air_demand,products,total_product_mol,products_normalised
    
    product_species = ['NITROGEN',
                       'CARBON DIOXIDE',
                       'WATER VAPOR',
                       'HYDROGEN CHLORIDE',
                       'CARBON MONOXIDE',
                       'SOOT']
    
    species_invoke2, new_oxydizer = species_lump(oxydizer_label,
                                                 airComponents,
                                                 airVolFrac,
                                                 speciesInvoke)
    print("species_invoke2")
    print(species_invoke2)

    species_invoke3, new_product = species_lump(product_label,
                                                product_species,
                                                products_normalised,
                                                species_invoke2)
    print("species_invoke2")
    print(species_invoke2)
    
    print("species_invoke3")
    print(species_invoke3)

    reaction_para = [fuel_label,
                     oxydizer_label,
                     product_label,
                     1,
                     air_demand,
                     total_product_mol]
    new_reac = creat_reac_input(reaction_para[0], reaction_para)
    
    return species_invoke3, new_oxydizer, new_product, new_reac

In [16]:
#    PRODUCTS
#    Molecular Weight (g/mol)            28.57721
#    Ambient Density (kg/m^3)            1.188
#    Initial Mass Fraction               0.000
#    Enthalpy of Formation (J/kg)     -2.84E+06

#    Sub Species                    Mass Fraction     Mole Fraction
#    NITROGEN                       7.016941E-01      7.158167E-01
#    CARBON DIOXIDE                 2.175212E-01      1.412456E-01
#    CARBON MONOXIDE                0.000000E+00      0.000000E+00
#    WATER VAPOR                    6.646607E-02      1.054335E-01
#    SOOT                           1.431862E-02      3.750416E-02

In [17]:
#    AIR
#    Molecular Weight (g/mol)            28.76564
#    Ambient Density (kg/m^3)            1.196
#    Initial Mass Fraction               1.000
#    Enthalpy of Formation (J/kg)     -7.44E+04

#    Sub Species                    Mass Fraction     Mole Fraction
#    NITROGEN                       7.630774E-01      7.835682E-01
#    OXYGEN                         2.311814E-01      2.078229E-01
#    CARBON DIOXIDE                 5.919362E-04      3.869034E-04
#    WATER VAPOR                    5.149269E-03      8.222023E-03

In [18]:
#    Fuel                              Heat of Combustion (kJ/kg)
#    TOLUENE                                           34963.6292

#    Stoichiometry

#    Primitive Species
#    Species ID                                 Stoich. Coeff.
#    TOLUENE                                            -1.000000
#    NITROGEN                                           -0.000000
#    OXYGEN                                             -7.609532
#    CARBON DIOXIDE                                      5.647113
#    WATER VAPOR                                         3.924840
#    SOOT                                                1.503208

#    Tracked (Lumped) Species
#    Species ID                                 Stoich. Coeff.
#    AIR                                            -36.615475
#    TOLUENE                                         -1.000000
#    PRODUCTS                                        40.081103

In [19]:
#  Mass Fraction Transformation Matrix to Convert Species Mixtures 
# (Columns) to Primitive Species (Rows)

#                          AIR       TOLUENE   PRODUCTS  WATER VA
#    TOLUENE               0.000000  1.000000  0.000000  0.000000
#    NITROGEN              0.763077  0.000000  0.701694  0.000000
#    OXYGEN                0.231181  0.000000  0.000000  0.000000
#    CARBON DIOXIDE        0.000592  0.000000  0.217521  0.000000
#    CARBON MONOXIDE       0.000000  0.000000  0.000000  0.000000
#    WATER VAPOR           0.005149  0.000000  0.066466  1.000000
#    SOOT                  0.000000  0.000000  0.014319  0.000000

In [20]:
transform_matrix = np.array([[0.000000, 1.000000, 0.000000, 0.000000],
                             [0.763077, 0.000000, 0.701694, 0.000000],
                             [0.231181, 0.000000, 0.000000, 0.000000], 
                             [0.000592, 0.000000, 0.217521, 0.000000], 
                             [0.000000, 0.000000, 0.000000, 0.000000], 
                             [0.005149, 0.000000, 0.066466, 1.000000],
                             [0.000000, 0.000000, 0.014319, 0.000000]])

In [21]:
np.sum(transform_matrix, axis=0)

array([0.999999, 1.      , 1.      , 1.      ])